In [116]:
import numpy as np
import pandas as pd
from scipy import stats

In [117]:
# The quantile function is simply the inverse CDF of a distribution
stats.norm.ppf(0.9999)

3.719016485455709

In [118]:
# Value at Risk
def var_para(distribution, alpha=0.05, **kwargs):
    var = distribution.ppf(q=alpha,**kwargs)
    return var

In [149]:
# Expected Shortfall
def ES_para( distribution, alpha, **kwargs):
    level = 1 - alpha
    VAR = var_para(distribution,alpha=alpha,**kwargs)
    if isinstance(distribution, stats.rv_discrete):
        First_term = distribution.expect(lambda x: -x, ub=VAR, conditional=True, args=(kwargs.values()) ) # mind the order of value in "kwargs"
    else: First_term = distribution.expect(lambda x: -x, ub=VAR, conditional=True,**kwargs ) # Thanks to an amazing library - scipy
    alpha_hat = 1 - distribution.cdf(distribution.ppf(1-alpha,**kwargs ),**kwargs )
    Second_term = -VAR*( alpha_hat - alpha )
    
    ES = -(First_term + Second_term)
    return ES

In [120]:
print("The value at risk of the standard normal distribution at 95% confidentblevel is", var_para(stats.norm,alpha=0.05))
print("The expected shortfall of the standard normal distribution at 95% confidentblevel is", ES_para(stats.norm,alpha=0.05))

The value at risk of the standard normal distribution at 95% confidentblevel is -1.6448536269514729
The expected shortfall of the standard normal distribution at 95% confidentblevel is -2.0627128078189143


## Let's compare

The parametric method and the non-parametric method
- First, simulate random numbers with a specific distribution
- use the non-parametric method with the numbers and compare with the parametric method

In [121]:
# non parametric

# V@R
def var_non_para(data, alpha):
    data = np.array(data)
    sorted_data = np.sort(data)
    
    arange_data = np.arange(len(data))
    percentile_sorted_data = arange_data/(len(data)-1)

    if max(percentile_sorted_data) <= alpha: # max(percentile_sorted_data) is 1
        integer = len(data) - 1
        partial = 0
        sorted_data = np.append(sorted_data, sorted_data[-1])

    elif min(percentile_sorted_data) >= alpha:
        integer = 0 # This is because we do not know the minimum value, so we assume that the minimum value is the smallest sample
        partial = 0

    else:
        for i in range(1,len(data)):
            if (percentile_sorted_data[i] > alpha):
                partial = (alpha-percentile_sorted_data[i-1])/(percentile_sorted_data[i] - percentile_sorted_data[i-1]) # assume that the risk between two samples is uniform
                integer = i - 1
                break
            
            elif (percentile_sorted_data[i] == alpha):
                integer = i 
                partial = 0
                break

    var = sorted_data[integer] + (sorted_data[integer + 1] - sorted_data[integer]) * partial
    return var

def ES_non_para(data, alpha):
    data = np.array(data)
    sorted_data = np.sort(data)

    var = var_non_para(data, alpha)

    prob_left_risk = len(sorted_data[sorted_data <= var])/len(sorted_data)
    for i in range(1,len(sorted_data)):
        if (sorted_data[i] > var):
            partial_prob_left_risk = (var - sorted_data[i-1])/(sorted_data[i] - sorted_data[i-1]) * ( 1 / len(sorted_data) )
            partial_prob_right_risk = (sorted_data[i] - var)/(sorted_data[i] - sorted_data[i-1]) * ( 1 / len(sorted_data) )
            break
        
        elif (sorted_data[i] == var):
            partial_prob_left_risk = 0
            partial_prob_right_risk = 0
            break
        
    numerator = (-np.sum(sorted_data[sorted_data <= var])/len(sorted_data) - (var * partial_prob_right_risk) ) + ( var * ( (prob_left_risk + partial_prob_left_risk) - alpha ) )
    return -numerator/(alpha)

### Exponential Distribution

In [122]:
# random numbers
np.random.seed(42)
rand_expo = np.random.exponential(scale=2, size=1000000)

VAR_expo_sim = var_non_para(rand_expo,alpha=0.1)
ES_expo_sim = ES_non_para(rand_expo,alpha=0.1)
print("The value at risk by the simulation method is ", VAR_expo_sim)
print("The expected shortfall by the simulation method is ", ES_expo_sim)

VAR_expo_para = var_para(stats.expon,scale=2,alpha=0.1)
ES_expo_para = ES_para(stats.expon,scale=2,alpha=0.1)
print("The value at risk by the parametric method is ", VAR_expo_para)
print("The expected shortfall by the parametric method is ", ES_expo_para)

The value at risk by the simulation method is  0.21109151861322278
The expected shortfall by the simulation method is  0.10405204792909421
The value at risk by the parametric method is  0.21072103131565262
The expected shortfall by the parametric method is  0.1035107181591266


### Normal Distribution

In [123]:
rand_norm = np.random.normal(loc=5,scale=1, size=1000000)

VAR_norm_sim = var_non_para(rand_norm,alpha=0.1)
ES_norm_sim = ES_non_para(rand_norm,alpha=0.1)
print("The value at risk by the simulation method is ", VAR_norm_sim)
print("The expected shortfall by the simulation method is ", ES_norm_sim)

VAR_norm_para = var_para(stats.norm, loc=5, scale=1, alpha=0.1)
ES_norm_para = ES_para(stats.norm, loc=5, scale=1, alpha=0.1)
print("The value at risk by the parametric method is ", VAR_norm_para)
print("The expected shortfall by the parametric method is ", ES_norm_para)

The value at risk by the simulation method is  3.714523869376157
The expected shortfall by the simulation method is  3.24099416464966
The value at risk by the parametric method is  3.7184484344553996
The expected shortfall by the parametric method is  3.245016680675133


### Binomial Distribution

In [151]:
rand_bin = np.random.binomial(n=300,p=0.32,size=1000000)

VAR_bin_sim = var_non_para(rand_bin,alpha=0.1)
ES_bin_sim = ES_non_para(rand_bin,alpha=0.1)
print("The value at risk by the simulation method is ", VAR_bin_sim)
print("The expected shortfall by the simulation method is ", ES_bin_sim)

VAR_bin_para = var_para(stats.binom, n=300, p=0.32, alpha=0.1)
ES_bin_para = ES_para(stats.binom, n=300, p=0.32, alpha=0.1)
print("The value at risk by the parametric method is ", VAR_bin_para)
print("The expected shortfall by the parametric method is ", ES_bin_para)

The value at risk by the simulation method is  86.0
The expected shortfall by the simulation method is  81.95609
The value at risk by the parametric method is  86.0
The expected shortfall by the parametric method is  82.4170731911514
